<a href="https://colab.research.google.com/github/hari-ar/A-to-Z-Resources-for-Students/blob/master/CaseStudy_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This is the notebook for the case study for Module 1 EDA.

Imports for the analysis.

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

Load the file.

In [ ]:
imported_data = pd.read_csv("/content/loan.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/loan.csv'

In [ ]:
imported_data.head()

In [ ]:
imported_data.shape


In [ ]:
imported_data.columns[imported_data.isna().any()].tolist()

In [ ]:

print("Count where all values are NA is :" , imported_data.isnull().all(axis=0).sum())


54 of the columns have all empty records. Hence they are not useful for our analysis. So removing them from the dataset.

Drop the cells with all na values and check for columns with high missing data.

In [ ]:
dropped_na=imported_data.dropna(axis=1, how='all')
round(100*dropped_na.isnull().mean(),2)


**Analyze columns with hign number of NA and remove them**
* We notice mths_since_last_record, next_pymnt_d has high nulls and can be safely removed.

* mths_since_last_delinq is also on the higher side, but since that is a field with potential impact, keeping it for now.

In [ ]:
filtered = dropped_na.drop(['mths_since_last_record', 'next_pymnt_d'], axis=1)
filtered.head()



*   There are several columns which are single valued




In [ ]:
filtered.info()

* We notice some numeric columns are objects like term, int_rate. So, check what is causing that.

In [ ]:
filtered.term.unique()

* Since the data contains only 2 values 36 months and 60 months we can leave it as is, instead of converting them to numeric values.

In [ ]:
filtered.int_rate.unique()

Since the rate contains % we can convert this to numeric if we strip the percentage.

In [ ]:
filtered['int_rate']=filtered.int_rate.apply(lambda x: float(str(x).replace("%", "")))
filtered.int_rate.unique()

* This shows the rates vary from about 6 all the way upto 24 in some instances.

Let us check the variance of the columns, remove outliers or columns with same values.

In [ ]:
filtered.var().sort_values()

There are some columns with 0 variance they can be safely removed.

In [ ]:
filtered = filtered.drop(['delinq_amnt', 'tax_liens', 'chargeoff_within_12_mths', 'acc_now_delinq', 'policy_code', 'collections_12_mths_ex_med'], axis=1)


Next up, while analysing the loan_status we see there are some loans with status as current. We might not need those rows as we do not have the signal of any default.

In [ ]:
filtered = filtered[filtered.loan_status.str.lower() != 'current']
filtered.info()

In [ ]:
filtered.loan_status.unique()

**Analysis**

* Since the assignment is about, understanding how likely it is for a person to default, we can skip univariate analysis and focus on bi-variate analysis.

* For further analysis, lets split the data into two sets. ChargedOff and Fullypaid.

In [ ]:
chargedOff=filtered[filtered.loan_status == 'Charged Off']
fullyPaid=filtered[filtered.loan_status == 'Fully Paid']

Let's analyze the interest rates of Defaulters vs Non Defaulters

In [ ]:
plt.subplot(1,2,1)
sns.boxplot(y=chargedOff.int_rate)
plt.title("\n Interest rate for ChargedOff")
plt.ylabel('Interest rate (%)')


plt.subplot(1,2,2)
sns.boxplot(y=fullyPaid.int_rate)
plt.title("\n Interest rate for fullyPaid")
plt.ylabel('Interest rate (%)')

plt.show()

* It is observed that, charged off applicants have high median interest rate. One possible explanation is based on their credit score or risk. Which inturn resullts in high defualt.